_________CLICK   RUN ALL ↑   TO START THE APP____

-> Make sure to turn on Lock-in and Motor Stage

-> Home the stage before startup. Use speed less than 10 mm/s

-> Terahertz peak at ~166.6 mm. Translation must be about this value.

-> Rest of the values are standard values for THz signal acquisition. 

In [ ]:
import clr
import time
import threading
from tkinter import Tk, Label, Button, Entry, StringVar, messagebox, Toplevel
import matplotlib.pyplot as plt
import math
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from tkinter import filedialog
from matplotlib.figure import Figure
from System import Decimal  # Importing .NET Decimal for precise control
from pymeasure.instruments.srs import SR830

# Add references to Thorlabs Kinesis DLLs
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\ThorLabs.MotionControl.IntegratedStepperMotorsCLI.dll")
from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.IntegratedStepperMotorsCLI import *

class IntegratedControlApp:
    def __init__(self, master):
        self.master = master
        self.master.title("TERABYTE App")

        # Device serial number and SR830 adapter setup
        self.serial_no = "45871810"  # Replace with your actual device serial number
        self.lockin = SR830("GPIB::8")  # Replace with the correct address of your lock-in amplifier

        # Initialize StringVar attributes
        self.device_info_var = StringVar()
        self.position_var = StringVar()
        self.status_var = StringVar(value="Status: Ready")
        self.from_distance_var = StringVar(value="163")
        self.to_distance_var = StringVar(value="166")
        self.step_size_var = StringVar(value="0.01")
        self.wait_time_var = StringVar(value="100")
        self.home_speed_var = StringVar(value="10")
        self.stage_speed_var = StringVar(value="10")
        self.acceleration_var = StringVar(value="10")
        self.elapsed_time_var = StringVar(value="00:00")
        self.peak_to_peak_var = StringVar()  # For displaying the peak-to-peak voltage
        self.max_position_var = StringVar()  # For displaying the position of max R value
        self.scan_repeat = StringVar(value = "3")

        # GUI Layout
        Label(master, text="Device Info").grid(row=0, column=0, columnspan=3)
        Label(master, textvariable=self.device_info_var).grid(row=1, column=0, columnspan=3)

        Label(master, text="Current Position (mm):").grid(row=2, column=0)
        Label(master, textvariable=self.position_var).grid(row=2, column=1)

        Label(master, text="Status:").grid(row=3, column=0)
        Label(master, textvariable=self.status_var).grid(row=3, column=1)

        Label(master, text="Homing Velocity (<10 mm/s):").grid(row=4, column=0)
        self.home_velocity_entry = Entry(master, textvariable=self.home_speed_var)
        self.home_velocity_entry.grid(row=4, column=1)
        Button(master, text="Home Stage", command=self.home_device).grid(row=5, column=0, columnspan=3)

        Label(master, text="Scan repeat:").grid(row=4, column=2)
        self.scan_repeat_entry = Entry(master, textvariable=self.scan_repeat,width=3)
        self.scan_repeat_entry.grid(row=4, column=3)

        self.run = 1
        self.run_var = StringVar()
        self.run_var.set(f"Scan: {self.run}")

        self.scan_status_label = Label(master, textvariable=self.run_var)
        self.scan_status_label.grid(row=4, column=4)

        Label(master, text="From Position (mm):").grid(row=6, column=0)
        self.from_distance_entry = Entry(master, textvariable=self.from_distance_var)
        self.from_distance_entry.grid(row=6, column=1)
        Button(master, text="Move to From Position", command=self.move_to_from_position).grid(row=6, column=2)
        Label(master, textvariable=self.elapsed_time_var).grid(row=6, column=3, sticky="e")  # Elapsed time beside the button

        Label(master, text="To Position (mm):").grid(row=7, column=0)
        self.to_distance_entry = Entry(master, textvariable=self.to_distance_var)
        self.to_distance_entry.grid(row=7, column=1)

        Label(master, text="Step Size (mm):").grid(row=8, column=0)
        self.step_size_entry = Entry(master, textvariable=self.step_size_var)
        self.step_size_entry.grid(row=8, column=1)

        Label(master, text="Wait Time (ms):").grid(row=9, column=0)
        self.wait_time_entry = Entry(master, textvariable=self.wait_time_var)
        self.wait_time_entry.grid(row=9, column=1)

        Label(master, text="Stage Speed (mm/s):").grid(row=10, column=0)
        self.stage_speed_entry = Entry(master, textvariable=self.stage_speed_var)
        self.stage_speed_entry.grid(row=10, column=1)

        Label(master, text="Acceleration (mm/s²):").grid(row=11, column=0)  # Acceleration input
        self.acceleration_entry = Entry(master, textvariable=self.acceleration_var)
        self.acceleration_entry.grid(row=11, column=1)

        Button(master, text="Start Measurement", command=self.start_measurement).grid(row=12, column=0, columnspan=2)
        Button(master, text="Stop Measurement", command=self.stop_measurement).grid(row=12, column=2)
        Button(master, text="Reset Graph", command=self.reset_graph).grid(row=13, column=0, columnspan=3)

        # Adjusted placement of Peak-to-Peak Voltage and Position of Max R below the graph
        Label(master, text="Vpp:").grid(row=12, column=3, columnspan=2, sticky="w")
        Label(master, textvariable=self.peak_to_peak_var).grid(row=12, column=4, columnspan=2, sticky="w")

        Label(master, text="R Max(mm):").grid(row=13, column=3, columnspan=2, sticky="w")
        Label(master, textvariable=self.max_position_var).grid(row=13, column=4, columnspan=2, sticky="w")

        # Initialize the graph window immediately
        self.create_graph_window()

        # Initialize Device
        self.initialize_device()

        # Start position monitoring thread
        self.monitoring = True
        self.position_monitor_thread = threading.Thread(target=self.update_position)
        self.position_monitor_thread.start()

        self.is_running = False  # Flag to control measurement

        # Data for plotting
        self.position_data, self.x_data, self.r_data = [], [], []

    def create_graph_window(self):
        """Open a new window for plotting the graphs at the start of the app."""
        self.new_window = Toplevel(self.master)
        self.new_window.title("Measurement Plots")

        # Create a new figure for the new window
        self.fig = Figure(figsize=(10, 5), dpi=100)
        self.ax1 = self.fig.add_subplot(121)
        self.ax2 = self.fig.add_subplot(122)
        self.ax1_twin = self.ax1.twiny()

        # Set up the graph canvas
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.new_window)
        self.canvas.get_tk_widget().pack()

        # Add the Matplotlib Navigation Toolbar
        self.toolbar = NavigationToolbar2Tk(self.canvas, self.new_window)
        self.toolbar.update()
        self.toolbar.pack(side="bottom", fill="x")
        
        Button(self.new_window, text="Save Data", command=self.save_data_as_text).pack()

        self.canvas.draw()

    def save_data_as_text(self):
        """Save the X (in ps) vs voltage data to a text file after modifying the mm column as specified."""
        # Ask user for the path to save the file
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        
        if file_path:
            # Speed of light in vacuum (m/s)
            c = 299792458

            # # Step size from user input (in mm)
            # step_size = float(self.step_size_var.get())

            # # Modify the position data: first value is zero, subsequent entries increment by step size
            # modified_position_data = [0]  # Start with zero
            # for i in range(1, len(self.position_data)):
            #     modified_position_data.append(modified_position_data[i-1] + step_size)

            # # Sort the modified position data in decreasing order
            # sorted_position_data = sorted(modified_position_data, reverse=True)

            sorted_position_data = self.position_data

            # Save the sorted position (converted to ps) and X voltage data into the file
            with open(file_path, 'w') as file:
                file.write("Time (ps)\tX Voltage (V)\n")  # Header

                for pos_mm, x_volt in zip(sorted_position_data, self.x_data):
                    # Convert mm to ps: ps = (mm * 2 * 1e9 / c)
                    time_ps = pos_mm * 2 * 1e9 / c
                    
                    # Write to file
                    file.write(f"{time_ps}\t{x_volt}\n")
            
            messagebox.showinfo("Success", f"Data saved to {file_path}")

    def update_graph(self):
        """Update the graph during the measurement."""
        self.ax1.clear()
        self.ax1_twin.clear()
        self.ax2.clear()

        c = 299792458
        self.time_ps = np.array(self.position_data)*2.0*1e9/c

        self.ax1.plot(self.position_data, self.x_data, linestyle='-', label='X Voltage vs Position')
        line = self.ax1_twin.plot(self.time_ps,self.x_data)
        line[0].set_visible(False)
        self.ax2.plot(self.position_data, self.r_data, linestyle='-', label='R Voltage vs Position')

        from matplotlib.ticker import EngFormatter
        # Set the y-axis to use engineering notation
        formatter = EngFormatter()  # Use engineering notation
        self.ax1.yaxis.set_major_formatter(formatter)
        self.ax1.yaxis.set_major_formatter(lambda x, pos: f'{formatter(x, pos)} V')  # Add 'V' to each label
        self.ax2.yaxis.set_major_formatter(formatter)
        self.ax2.yaxis.set_major_formatter(lambda x, pos: f'{formatter(x, pos)} V')  # Add 'V' to each label

        self.ax1.set_xlabel("Position (mm)")
        self.ax1_twin.set_xlabel("Delay (ps)")
        self.ax1_twin.xaxis.set_label_position('top')
        self.ax1.set_ylabel("X Voltage")
        self.ax2.set_xlabel("Position (mm)")
        self.ax2.set_ylabel("R Voltage")
        self.ax2.yaxis.set_label_position('right')

        self.canvas.draw()

    def initialize_device(self):
        """Initialize the Thorlabs LTS Stage device."""
        try:
            DeviceManagerCLI.BuildDeviceList()
            time.sleep(1)
            self.device = LongTravelStage.CreateLongTravelStage(self.serial_no)
            self.device.Connect(self.serial_no)

            if not self.device.IsSettingsInitialized():
                self.device.WaitForSettingsInitialized(10000)
                self.device.LoadMotorConfiguration(self.serial_no)

            self.device.StartPolling(250)
            time.sleep(0.25)
            self.device.EnableDevice()
            time.sleep(0.25)

            device_info = self.device.GetDeviceInfo()
            self.device_info_var.set(f"{device_info.Description} (SN: {device_info.SerialNumber})")
            print("Device initialized successfully.")

        except Exception as e:
            messagebox.showerror("Error", f"Error initializing delay stage: {e}")

    def update_position(self):
        """Continuously update the current position of the stage."""
        while self.monitoring:
            try:
                current_position = float(str(self.device.Position))
                self.position_var.set(f"{current_position:.6f}")
            except Exception as e:
                self.position_var.set("Error")
            time.sleep(0.1)

    def home_device(self):
        """Home the LTS stage with a specified velocity."""
        try:
            velocity = float(self.home_speed_var.get())
            home_params = self.device.GetHomingParams()
            home_params.Velocity = Decimal(velocity)
            self.device.SetHomingParams(home_params)

            self.status_var.set("Status: Homing...")
            self.master.update()

            self.device.Home(60000)
            self.status_var.set("Status: Ready")
            print("Device Homed Successfully")
        except Exception as e:
            messagebox.showerror("Error", f"Error homing delay stage: {e}")

    def move_to_from_position(self):
        """Move the stage to the 'From Position' at the specified speed."""
        try:
            from_position = float(self.from_distance_var.get())
            velocity = float(10)  #Move to position speed fixed at 10 mm/s. Dont think we need to change that
            acceleration = float(self.acceleration_var.get())

            move_params = self.device.GetVelocityParams()
            move_params.MaxVelocity = Decimal(velocity)
            move_params.Acceleration = Decimal(acceleration)
            self.device.SetVelocityParams(move_params)

            self.status_var.set("Status: Moving to From Position...")
            self.master.update()

            self.device.MoveTo(Decimal(from_position), 60000)
            self.status_var.set("Status: Ready")
            print("Reached From Position Successfully")

        except Exception as e:
            messagebox.showerror("Error", f"Error moving to from position: {e}")

    def start_measurement(self):
        """Start the measurement process: Move the stage in steps and collect data."""
        try:
            from_position = float(self.from_distance_var.get())
            to_position = float(self.to_distance_var.get())
            step_size = abs(float(self.step_size_var.get()))
            wait_time = float(self.wait_time_var.get()) / 1000
            speed = float(self.stage_speed_var.get())
            acceleration = float(self.acceleration_var.get())

            self.is_running = True
            self.status_var.set("Status: Measuring...")
            self.master.update()

            move_params = self.device.GetVelocityParams()
            move_params.MaxVelocity = Decimal(speed)
            move_params.Acceleration = Decimal(acceleration)
            self.device.SetVelocityParams(move_params)

            self.start_time = time.time()

            threading.Thread(target=self.measure, args=(from_position, to_position, step_size, wait_time)).start()
            threading.Thread(target=self.update_elapsed_time).start()
            

        except Exception as e:
            messagebox.showerror("Error", f"Error starting measurement: {e}")

    def measure(self, from_position, to_position, step_size, wait_time):
        """Measure and update dynamically at each step."""
        try:
            self.run =1
            
            while self.run <= int(self.scan_repeat.get()):

                # Update label
                self.run_var.set(f"Scan: {self.run}")
                # Force UI refresh (needed when in a background thread)
                self.master.update_idletasks()
                if to_position >= from_position: step_size = abs(step_size)
                else : step_size = -abs(step_size)
                current_position = from_position

                lower_bound = min(from_position, to_position)
                upper_bound = max(from_position, to_position)
                while self.is_running and lower_bound <= current_position <= upper_bound:
                    self.device.MoveTo(Decimal(current_position), 60000)
                    time.sleep(wait_time)

                    x_voltage, r_value = self.read_lockin_values()
                    self.position_data.append(current_position)
                    self.x_data.append(x_voltage)
                    self.r_data.append(r_value)

                    # Update the graph during measurement
                    self.update_graph()

                    current_position += step_size
                self.run+=1
                from_position_2 = to_position
                to_position = from_position
                from_position = from_position_2

            self.status_var.set("Status: Measurement complete.")
            self.calculate_results()
            self.is_running = False

        except Exception as e:
            messagebox.showerror("Error", f"Error during measurement: {e}")

    def calculate_results(self):
        """Calculate peak-to-peak voltage and position of max R value."""
        if self.x_data:
            peak_to_peak_voltage = max(self.x_data) - min(self.x_data)
            self.peak_to_peak_var.set(f"{peak_to_peak_voltage*1000:.3f} mV")
        
        if self.r_data:
            max_r_value = max(self.r_data)
            max_position_index = self.r_data.index(max_r_value)
            max_position = self.position_data[max_position_index]
            self.max_position_var.set(f"{max_position:.6f} mm")
        
    def stop_measurement(self):
        """Stop the measurement process."""
        self.is_running = False
        self.status_var.set("Status: Measurement stopped.")
        print("Measurement stopped by the user.")

    def reset_graph(self):
        """Reset the graph and time."""
        self.position_data.clear()
        self.x_data.clear()
        self.r_data.clear()
        self.elapsed_time_var.set("00:00")
        self.time_ps = np.array([])
        self.update_graph()  # Reset the graph display
        print("Graph has been reset.")

    def update_elapsed_time(self):
        """Update the elapsed time in minutes and seconds format, stops when measurement is complete."""
        while self.is_running:
            elapsed_time = time.time() - self.start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            self.elapsed_time_var.set(f"{minutes:02d}:{seconds:02d}")
            time.sleep(1)
    
        # Stop the time at the completion value
        elapsed_time = time.time() - self.start_time
        minutes, seconds = divmod(int(elapsed_time), 60)
        self.elapsed_time_var.set(f"{minutes:02d}:{seconds:02d}")

    def read_lockin_values(self):
        """Reads X and R values from the lock-in amplifier using snap."""
        try:
            x_voltage, r_value = self.lockin.snap('X', 'R')
            return x_voltage, r_value
        except Exception as e:
            messagebox.showerror("Error", f"Error reading lock-in values: {e}")
            return 0, 0

    def close(self):
        """Close the application and clean up resources."""
        self.is_running = False
        self.monitoring = False
        if self.device:
            self.device.StopPolling()
            self.device.Disconnect()
        self.master.destroy()

if __name__ == "__main__":
    root = Tk()
    app = IntegratedControlApp(root)
    root.protocol("WM_DELETE_WINDOW", app.close)
    root.mainloop()


#Credits ChatGPT 4o with Rohith